In [7]:
import requests
import cv2
import numpy as np
from PIL import Image, ImageDraw
import math
import paho.mqtt.client as mqtt
import json

# Datos del robot (cm)
L1 = 15.0  # l1
L2 = 24.0  # l2
base_height_b = 18.0

# URL de la cámara ESP32-CAM 
url = "http://192.168.141.115:81/stream"

API_URL = "https://detect.roboflow.com/yolov9-eggs/1"
API_KEY = "xpjWjOAvyietZJ6FJckO"
PARAMS = {
    "api_key": API_KEY,
    "confidence": 40,
    "overlap": 30
}

# Configuración MQTT
MQTT_HOST = "192.168.141.204"
MQTT_PORT = 1883
MQTT_TOPIC = "casa/salon/coordenadas"
MQTT_TOPIC_FOTO = "casa/salon/foto"

# Variable global para controlar la captura de foto
tomar_foto = False

def inverse_kinematics_rrr(px, py, pz, l1, l2, base_height_b, elbow_up=True):
    pz_eff = pz - base_height_b

    theta1_rad = np.arctan2(py, px)
    theta1_deg = np.degrees(theta1_rad)

    D_squared = px**2 + py**2 + pz_eff**2

    denom_c3 = 2 * l1 * l2
    if denom_c3 == 0:
        print("Error: l1 o l2 no pueden ser cero.")
        return None

    cos_theta3 = (D_squared - l1**2 - l2**2) / denom_c3
    if not -1 <= cos_theta3 <= 1:
        print(f"Error: Fuera de alcance. cos(theta3) = {cos_theta3:.4f}")
        return None

    sin_theta3 = np.sqrt(max(0, 1 - cos_theta3**2))
    if not elbow_up:
        sin_theta3 = -sin_theta3

    theta3_rad = np.arctan2(sin_theta3, cos_theta3)
    theta3_deg = -np.degrees(theta3_rad)  # Signo invertido para motor

    K1 = l1 + l2 * np.cos(theta3_rad)
    K2 = l2 * np.sin(theta3_rad)

    R_p = px * np.cos(theta1_rad) + py * np.sin(theta1_rad)
    denom_t2 = K1**2 + K2**2
    if denom_t2 == 0:
        print("Error: Singularidad en el cálculo de theta2.")
        return None

    sin_theta2 = (pz_eff * K1 - R_p * K2) / denom_t2
    cos_theta2 = (R_p * K1 + pz_eff * K2) / denom_t2

    theta2_rad = np.arctan2(sin_theta2, cos_theta2)
    theta2_deg = np.degrees(theta2_rad)

    return theta1_deg, theta2_deg, theta3_deg

def procesar_imagen(image_path):
    image = open(image_path, "rb").read()
    response = requests.post(API_URL, params=PARAMS, files={"file": image})
    detections = response.json()

    print("Detecciones:", detections)

    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    cantidad_detectados = 0
    if "predictions" in detections and detections['predictions']:
        cantidad_detectados = len(detections['predictions'])  # NUEVO

    if "predictions" in detections:
        for prediction in detections['predictions']:
            x, y, w, h = prediction['x'], prediction['y'], prediction['width'], prediction['height']
            class_name = prediction['class']
            confidence = prediction['confidence']

            draw.rectangle([x - w / 2, y - h / 2, x + w / 2, y + h / 2], outline="red", width=2)

            centro_imagen_x = 640 // 2
            tolerancia = 20
            if x < centro_imagen_x - tolerancia:
                print("Objeto a la DERECHA del centro → base mover DERECHA")
                sentido_base = "derecha"
            elif x > centro_imagen_x + tolerancia:
                print("Objeto a la IZQUIERDA del centro → base mover IZQUIERDA")
                sentido_base = "izquierda"
            else:
                print("Objeto CENTRADO → no mover base")
                sentido_base = "centrado"

            # Convertir pixeles a cm (según tu calibración)
            ejex = (x * 0.0328) + 0.5
            ejey = (y * 0.0327) + 0.3
            pz_desired = 0.0

            print(f"Centro del rectángulo: ({ejex:.2f} cm, {ejey:.2f} cm)")

            result = inverse_kinematics_rrr(ejex, ejey, pz_desired, L1, L2, base_height_b, elbow_up=True)
            if result is None:
                print("No se pudo calcular cinemática inversa para este punto.")
                continue

            theta1_deg, theta2_deg, theta3_deg = result

            # Ajustar signo theta1 según lado
            if sentido_base == "derecha":
                theta1_deg = -abs(theta1_deg)
            elif sentido_base == "izquierda":
                theta1_deg = abs(theta1_deg)
            else:
                theta1_deg = 0

            print(f"Ángulo base (theta1): {theta1_deg:.2f}°")
            print(f"Ángulo hombro (theta2): {theta2_deg:.2f}°")
            print(f"Ángulo codo (theta3): {theta3_deg:.2f}°")

            pasos1 = np.round((theta1_deg * 200) / 180)
            pasos2 = np.round((theta2_deg * 1000) / 180)
            pasos3 = np.round((theta3_deg * 1000) / 180)

            print(f"Pasos motor 1 (base): {pasos1:.0f}")
            print(f"Pasos motor 2 (hombro): {pasos2:.0f}")
            print(f"Pasos motor 3 (codo): {pasos3:.0f}")

            draw.text((x - w / 2, y - h / 2), f"{class_name} ({confidence:.2f}%)", fill="red")

            # Enviar datos via MQTT
            coordenadas = {
                "theta1": round(theta1_deg+5,2),#5
                "theta2": round(theta2_deg,2),#-9
                "theta3": round(theta3_deg,2)#-9
            }
            mqtt_payload = json.dumps(coordenadas)
            mqtt_client.publish(MQTT_TOPIC, mqtt_payload)
            print(f"Publicado en MQTT {MQTT_TOPIC}: {mqtt_payload}")
      # Publicar conteo final de huevos (incluso si es 0)
    mqtt_client.publish("casa/salon/conteohuevos", f"Huevos detectados: {cantidad_detectados}")  # NUEVO
    print(f"Publicado en MQTT casa/salon/conteohuevos: Huevos detectados: {cantidad_detectados}")  # NUEVO
    img.show()

def on_mqtt_message(client, userdata, msg):
    global tomar_foto
    topic = msg.topic
    payload = msg.payload.decode()
    print(f"MQTT mensaje recibido en topic {topic}: {payload}")
    if topic == MQTT_TOPIC_FOTO and payload == "ON":
        tomar_foto = True

mqtt_client = mqtt.Client()
mqtt_client.on_message = on_mqtt_message
mqtt_client.connect(MQTT_HOST, MQTT_PORT, 60)
mqtt_client.subscribe(MQTT_TOPIC_FOTO)
mqtt_client.loop_start()

cap = cv2.VideoCapture(url)

if not cap.isOpened():
    print("Error: No se pudo abrir la transmisión de video.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo recibir el frame.")
        break

    cv2.imshow('ESP32-CAM', frame)

    if tomar_foto:
        captured_image_path = "captura_esp32.jpg"
        cv2.imwrite(captured_image_path, frame)
        print(f"Imagen capturada y guardada como {captured_image_path}")
        procesar_imagen(captured_image_path)
        tomar_foto = False

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
mqtt_client.loop_stop()
mqtt_client.disconnect()


C:\Users\jarar\AppData\Local\Temp\ipykernel_9172\1179841850.py:167: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


MQTT mensaje recibido en topic casa/salon/foto: ON
Imagen capturada y guardada como captura_esp32.jpg
Detecciones: {'inference_id': '0bdc2969-77de-4b07-ad59-f3d144b70223', 'time': 0.2563383500000782, 'image': {'width': 640, 'height': 480}, 'predictions': [{'x': 134.5, 'y': 280.5, 'width': 199.0, 'height': 167.0, 'confidence': 0.9654804468154907, 'class': 'egg', 'points': [{'x': 109.0, 'y': 197.25}, {'x': 108.0, 'y': 198.0}, {'x': 107.0, 'y': 198.0}, {'x': 106.0, 'y': 198.75}, {'x': 105.0, 'y': 198.75}, {'x': 103.0, 'y': 200.25}, {'x': 102.0, 'y': 200.25}, {'x': 101.0, 'y': 201.0}, {'x': 97.0, 'y': 201.0}, {'x': 96.0, 'y': 201.75}, {'x': 95.0, 'y': 201.75}, {'x': 93.0, 'y': 203.25}, {'x': 92.0, 'y': 203.25}, {'x': 91.0, 'y': 204.0}, {'x': 89.0, 'y': 204.0}, {'x': 88.0, 'y': 204.75}, {'x': 88.0, 'y': 205.5}, {'x': 86.0, 'y': 207.0}, {'x': 85.0, 'y': 207.0}, {'x': 81.0, 'y': 210.0}, {'x': 80.0, 'y': 210.0}, {'x': 79.0, 'y': 210.75}, {'x': 79.0, 'y': 211.5}, {'x': 78.0, 'y': 212.25}, {'x':

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [ ]:
!pip install paho-mqtt


  Using cached paho_mqtt-2.1.0-py3-none-any.whl.metadata (23 kB)
Using cached paho_mqtt-2.1.0-py3-none-any.whl (67 kB)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
